In [1]:
import numpy as np
import matplotlib.pyplot as plt
from exact_solvers import euler
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim, nn, utils, Tensor
import torch.nn.functional as F
import lightning.pytorch as pl
import mlflow.pytorch
from sklearn.preprocessing import StandardScaler

*** Warning: JSAnimation not found
*** Warning: JSAnimation not found


In [2]:
State = euler.Primitive_State
scaler = StandardScaler()

stencil = 5
offset = stencil//2

inputs = []
labels = []

gammas = np.linspace(1.1, 2.0, 10)
times = np.linspace(1e-10, 1.0, 100)
PLs = np.linspace(3, 13, 5)
# PRs = np.linspace(3, 13, 5)
PRs = [0.1]
# grid_sizes = np.power(2, np.arange(6, 12))
grid_sizes = [512]

for gs in grid_sizes:
    print(gs)
    N = gs
    a = -2.0
    b = 2.0

    x = np.linspace(a, b, N)
    dx = (b - a)/N
    
    div = np.zeros(N)
    
    for PL in PLs:
        for PR in PRs:
            left_state  = State(Density = 3.,
                                Velocity = 0.,
                                Pressure = PL)
            right_state = State(Density = 1.,
                                Velocity = 0.,
                                Pressure = PR)
            
            q_l = euler.primitive_to_conservative(*left_state)
            q_r = euler.primitive_to_conservative(*right_state)
            
            
            for gamma in gammas:
            
                states, speeds, reval, wave_types = euler.exact_riemann_solution(q_l, q_r, gamma=gamma)    
                
                for t in times:
                    rho, u, P = euler.cons_to_prim(reval(x/t))

                    if (PL > PR):
                        
                        raref_l, raref_r = speeds[0][0] * t, speeds[0][1] * t
                        contact = speeds[1] * t
                        shock = speeds[2] * t
                    elif (PR > PL):
                        raref_l, raref_r = speeds[2][0] * t, speeds[2][1] * t
                        contact = speeds[1] * t
                        shock = speeds[0] * t
                    else:
                        continue
                
                    div[1:-2] = (u[2:-1] - u[0:-3])/(2 * dx)
                    rho, u, P, div = StandardScaler().fit_transform(rho.reshape(-1,1)).flatten(), StandardScaler().fit_transform(u.reshape(-1,1)).flatten(), StandardScaler().fit_transform(P.reshape(-1,1)).flatten(),StandardScaler().fit_transform(div.reshape(-1,1)).flatten()
                    
                    #shock_loc = np.logical_and(x > (shock - eps),x < (shock + eps))
                    for i in range(offset, N - offset):
                        input = np.zeros((2,stencil))
                        input[0, :] = rho[(i-offset):(i + offset + 1)]
                        #input[0, :] -= np.mean(rho[(i-offset):(i + offset + 1)])
                        # input[1, :] = u[(i-offset):(i + offset + 1)]
                        #input[1, :] -= np.mean(u[(i-offset):(i + offset + 1)])
                        input[1, :] = P[(i-offset):(i + offset + 1)]
                        #input[2, :] -= np.mean(P[(i-offset):(i + offset + 1)])
                        # input[3, :] = div[(i-offset):(i + offset + 1)]
                        #input[3, :] -= np.mean(div[(i-offset):(i + offset + 1)])
                        inputs.append(input)
                        #if np.any(x[(i-offset):(i + offset + 1)] >= shock) and np.any(x[(i-offset):(i + offset + 1)] <= shock):
                        if np.any(x[(i-offset):(i + offset + 1)] > shock) and np.any(x[(i-offset):(i + offset + 1)] < shock):
                            labels.append(1)
                        elif np.any(x[(i-offset):(i + offset + 1)] > contact) and np.any(x[(i-offset):(i + offset + 1)] < contact):
                            labels.append(2)
                        else:
                            labels.append(0)
            

# random.seed(42)
# indices_0 = [i for i, label in enumerate(labels) if label == 0]
# indices_1 = [i for i, label in enumerate(labels) if label == 1]

# # Randomly select a subset of 0s with the same size as the number of 1s
# subset_indices_0 = random.sample(indices_0, len(indices_1))

# # Create new balanced lists
# balanced_inputs = [inputs[i] for i in subset_indices_0 + indices_1]
# balanced_labels = [labels[i] for i in subset_indices_0 + indices_1]

512


In [3]:

gammas = np.linspace(1.1, 2.0, 10)
times = np.linspace(1e-10, 1.0, 100)
PRs = np.linspace(3, 13, 5)
# PRs = np.linspace(3, 13, 5)
PLs = [0.1]
#grid_sizes = np.power(2, np.arange(6, 12))
grid_sizes = [512]

for gs in grid_sizes:
    print(gs)
    N = gs
    a = -2.0
    b = 2.0

    x = np.linspace(a, b, N)
    dx = (b - a)/N
    
    div = np.zeros(N)
    
    for PL in PLs:
        for PR in PRs:
            left_state  = State(Density = 1.,
                                Velocity = 0.,
                                Pressure = PL)
            right_state = State(Density = 3.,
                                Velocity = 0.,
                                Pressure = PR)
            
            q_l = euler.primitive_to_conservative(*left_state)
            q_r = euler.primitive_to_conservative(*right_state)
            
            
            for gamma in gammas:
            
                states, speeds, reval, wave_types = euler.exact_riemann_solution(q_l, q_r, gamma=gamma)    
                
                for t in times:
                    rho, u, P = euler.cons_to_prim(reval(x/t))

                    if (PL > PR):
                        
                        raref_l, raref_r = speeds[0][0] * t, speeds[0][1] * t
                        contact = speeds[1] * t
                        shock = speeds[2] * t
                    elif (PR > PL):
                        raref_l, raref_r = speeds[2][0] * t, speeds[2][1] * t
                        contact = speeds[1] * t
                        shock = speeds[0] * t
                    else:
                        continue
                
                    div[1:-2] = (u[2:-1] - u[0:-3])/(2 * dx)
                    rho, u, P, div = StandardScaler().fit_transform(rho.reshape(-1,1)).flatten(), StandardScaler().fit_transform(u.reshape(-1,1)).flatten(), StandardScaler().fit_transform(P.reshape(-1,1)).flatten(),StandardScaler().fit_transform(div.reshape(-1,1)).flatten()

                    
                    #shock_loc = np.logical_and(x > (shock - eps),x < (shock + eps))
                    for i in range(offset, N - offset):
                        input = np.zeros((2,stencil))
                        input[0, :] = rho[(i-offset):(i + offset + 1)]
                        #input[0, :] -= np.mean(rho[(i-offset):(i + offset + 1)])
                        # input[1, :] = u[(i-offset):(i + offset + 1)]
                        #input[1, :] -= np.mean(u[(i-offset):(i + offset + 1)])
                        input[1, :] = P[(i-offset):(i + offset + 1)]
                        #input[2, :] -= np.mean(P[(i-offset):(i + offset + 1)])
                        # input[3, :] = div[(i-offset):(i + offset + 1)]
                        #input[3, :] -= np.mean(div[(i-offset):(i + offset + 1)])
                        inputs.append(input)
                        #if np.any(x[(i-offset):(i + offset + 1)] >= shock) and np.any(x[(i-offset):(i + offset + 1)] <= shock):
                        if np.any(x[(i-offset):(i + offset + 1)] > shock) and np.any(x[(i-offset):(i + offset + 1)] < shock):
                            labels.append(1)
                        elif np.any(x[(i-offset):(i + offset + 1)] > contact) and np.any(x[(i-offset):(i + offset + 1)] < contact):
                            labels.append(2)
                        else:
                            labels.append(0)
            

random.seed(42)
# Separate the indices based on their labels
indices_0 = [i for i, label in enumerate(labels) if label == 0]
indices_1 = [i for i, label in enumerate(labels) if label == 1]
indices_2 = [i for i, label in enumerate(labels) if label == 2]

# Determine the smallest count among the labels
min_count = min(len(indices_0), len(indices_1), len(indices_2))

# Randomly sample from each label to match the smallest count
subset_indices_0 = random.sample(indices_0, min_count)
subset_indices_1 = random.sample(indices_1, min_count)
subset_indices_2 = random.sample(indices_2, min_count)

# Combine the sampled indices
balanced_indices = subset_indices_0 + subset_indices_1 + subset_indices_2

# Shuffle the combined indices to mix the samples
random.shuffle(balanced_indices)

# Create new balanced lists
balanced_inputs = [inputs[i] for i in balanced_indices]
balanced_labels = [labels[i] for i in balanced_indices]

512


In [4]:
train_data = []
for i, l in zip(balanced_inputs, balanced_labels):
    train_data.append([torch.tensor(i.flatten(), dtype=torch.float32), torch.tensor(l, dtype=torch.float32)])
    
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]

        # Assuming the first element is the input tensor and the second element is the label tensor
        input_tensor = torch.Tensor(sample[0])
        label_tensor = torch.Tensor(sample[1])

        return input_tensor, label_tensor
        
dataset = CustomDataset(train_data)
seed = torch.Generator().manual_seed(42)
train_set, valid_set = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=seed)

In [5]:
torch.save(train_set, "../../../data/ds-2/train_set.pt")
torch.save(valid_set, "../../../data/ds-2/valid_set.pt")
print (len(train_set))
print (len(valid_set))

91133
22783


In [13]:
class Shock_Cell_Classifier(pl.LightningModule):
    def __init__(self, input_dim, lr = 1e-4, wd = 1.0e-5):
        super().__init__()
        self.save_hyperparameters()

        # self.fc1 = nn.Linear(self.hparams.input_dim, 16)
        # self.fc2 = nn.Linear(16, 2)
        
        self.fc1 = nn.Linear(self.hparams.input_dim, self.hparams.input_dim)
        self.fc2 = nn.Linear(self.hparams.input_dim, 3)
        # self.fc3 = nn.Linear(8, 2)

        self.dropout5 = nn.Dropout(0.50)
        self.dropout2 = nn.Dropout(0.20)

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        # nn.init.xavier_uniform_(self.fc3.weight)
        
        #Initialize bias terms to zero
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.bias, 0)
        # nn.init.constant_(self.fc3.bias, 0)

        self.loss_fn = nn.CrossEntropyLoss()

        # self.logger.log_hyperparams()
    def forward(self, x):
        # x = self.dropout2(x)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.dropout5(x)
        x = self.fc2(x)
        # x = F.leaky_relu(x)
        # x = self.fc3(x)

        return x

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        inputs, labels = batch
        
        output = self.forward(inputs)

        
        loss = self.loss_fn(output, labels.long())
        
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        inputs, labels = batch
        
        output = self.forward(inputs)
        
        val_loss = self.loss_fn(output, labels.long())
        self.log("val_loss", val_loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.wd)
        return optimizer


# init the classifier
classifier = Shock_Cell_Classifier(stencil*2)

In [ ]:
classifier.train()

batch_size = 16

train_loader = DataLoader(train_set, batch_size=batch_size)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

trainer = pl.Trainer(max_epochs=30,
                     accelerator="cpu")

mlflow.pytorch.autolog()
with mlflow.start_run() as run:
    # mlflow.pytorch.log_model(classifier, "model")
    mlflow.log_params({
            "batch_size": batch_size,
        })
    trainer.fit(model=classifier,
                train_dataloaders=train_loader,
                val_dataloaders=valid_loader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/Users/henry/euler-nn/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
2024/05/22 16:05:26 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2024/05/22 16:05:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/henry/euler-nn/.venv/lib/python3.11/site-packages/mlflow/pytorch/_lightning_autolog.py:463: UserWarning: Autologging is known to be compatible with p

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/Users/henry/euler-nn/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/henry/euler-nn/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

In [ ]:
classifier.eval()

State = euler.Primitive_State
left_state  = State(Density = 3.,
                    Velocity = 0.,
                    Pressure = 6.5)

right_state = State(Density = 1.,
                    Velocity = 0.,
                    Pressure = 1.)
# classifier.eval()
# left_state  = State(Density = 1.,
#                     Velocity = 0.,
#                     Pressure = 1.0)

# right_state = State(Density = 3.,
#                     Velocity = 0.,
#                     Pressure = 8.)

q_l = euler.primitive_to_conservative(*left_state)
q_r = euler.primitive_to_conservative(*right_state)
states, speeds, reval, wave_types = euler.exact_riemann_solution(q_l, q_r, gamma=1.4)

N = 256
a = -2.0
b = 2.0
x = np.linspace(a, b, N)
dx = (b - a)/N

t = 0.5
rho, u, P = euler.cons_to_prim(reval(x/t))
classes = np.zeros(N)
div = np.zeros(N)
div[1:-2] = (u[2:-1] - u[0:-3])/(2 * dx)
rho, u, P, div = StandardScaler().fit_transform(rho.reshape(-1,1)).flatten(), StandardScaler().fit_transform(u.reshape(-1,1)).flatten(), StandardScaler().fit_transform(P.reshape(-1,1)).flatten(),StandardScaler().fit_transform(div.reshape(-1,1)).flatten()


for i in range(offset, N-offset):
    input = np.zeros((2,stencil))
    input[0, :] = rho[(i-offset):(i + offset + 1)]
    #input[0, :] -= np.mean(rho[(i-offset):(i + offset + 1)])
    # input[1, :] = u[(i-offset):(i + offset + 1)]
    #input[1, :] -= np.mean(u[(i-offset):(i + offset + 1)])
    input[1, :] = P[(i-offset):(i + offset + 1)]
    #input[2, :] -= np.mean(P[(i-offset):(i + offset + 1)])
    # input[3, :] = div[(i-offset):(i + offset + 1)]
    #input[3, :] -= np.mean(div[(i-offset):(i + offset + 1)])
    input = input.flatten()
    input = torch.tensor(input, dtype=torch.float32)
    output = classifier(input)
    classes[i] = output.argmax()
    if classes[i] == 1:
        print(output)

fig, axs = plt.subplots(4,1, figsize=(8,8))
axs[0].plot(x, rho, linestyle="--")
axs[1].plot(x, u, linestyle="--")
axs[2].plot(x, P, linestyle="--")
axs[3].plot(x, div, linestyle="--")
for i in range(0, N):
    if (classes[i] == 1):
        axs[0].plot(x[i], rho[i], marker= 'o', markerfacecolor="none", markeredgecolor="red")
        axs[1].plot(x[i], u[i], marker= 'o', markerfacecolor="none", markeredgecolor="red")
        axs[2].plot(x[i], P[i], marker= 'o', markerfacecolor="none", markeredgecolor="red")
    elif (classes[i] == 2):
        axs[0].plot(x[i], rho[i], marker= 'o', markerfacecolor="none", markeredgecolor="green")
        axs[1].plot(x[i], u[i], marker= 'o', markerfacecolor="none", markeredgecolor="green")
        axs[2].plot(x[i], P[i], marker= 'o', markerfacecolor="none", markeredgecolor="green")
    else:
        axs[0].plot(x[i], rho[i], marker='o', markerfacecolor="none", markeredgecolor="blue")
        axs[1].plot(x[i], u[i], marker= 'o', markerfacecolor="none", markeredgecolor="blue")
        axs[2].plot(x[i], P[i], marker= 'o', markerfacecolor="none", markeredgecolor="blue")
plt.tight_layout()
plt.show()

In [ ]:
torch.save(classifier.state_dict(), "../../../models/reduced-contact/nn-red.pt")

In [ ]:
classifier.to_torchscript("../../../models/reduced-contact/script-nn-red.pt")

In [ ]:

np.savetxt("./w-b/reduced-contact/fc1w.txt", classifier.state_dict()["fc1.weight"].numpy())
np.savetxt("./w-b/reduced-contact/fc1b.txt", classifier.state_dict()["fc1.bias"].numpy())
np.savetxt("./w-b/reduced-contact/fc2w.txt", classifier.state_dict()["fc2.weight"].numpy())
np.savetxt("./w-b/reduced-contact/fc2b.txt", classifier.state_dict()["fc2.bias"].numpy())